In [1]:
# 第4章 深层神经网络

In [1]:
import tensorflow as tf
v = tf.constant([[1.0, 2.0, 3.0],[4.0, 5.0, 6.0]])
print(tf.clip_by_value(v, 2.5, 4.5).eval)    # clip_by_value 将结果限制在2.5和4.5之间

<bound method Tensor.eval of <tf.Tensor 'clip_by_value:0' shape=(2, 3) dtype=float32>>


In [12]:
v = tf.constant([[1.0, 2.0, 3.0]])
print(tf.log(v).eval)

<bound method Tensor.eval of <tf.Tensor 'Log_3:0' shape=(1, 3) dtype=float32>>


In [16]:
result = tf.log(v)

In [17]:
result.eval

<bound method Tensor.eval of <tf.Tensor 'Log_7:0' shape=(1, 3) dtype=float32>>

In [5]:
# 带正则化的损失函数
w = tf.constant([[1.0, -2.0], [3.2, 3.1]])
with tf.Session() as sess:
    print(sess.run(tf.contrib.layers.l1_regularizer(.5)(w)))    # L1正则
    print(sess.run(tf.contrib.layers.l2_regularizer(.5)(w)))

4.6499996
6.2125


In [4]:
v1 = tf.constant([1.0, 2.0, 3.0, 4.0])
v2 = tf.constant([4.0, 3.0, 2.0, 1.0])
sess = tf.InteractiveSession()
print(tf.greater(v1, v2).eval())
print(tf.where(tf.greater(v1, v2), v1, v2).eval())     # where函数根据第一个参数觉得选择输出大的那个，类似于c中的：？运算符。
sess.close()

[False False  True  True]
[4. 3. 3. 4.]


In [24]:
# 使用自定义损失函数
from numpy.random import RandomState
batch_size = 8
x = tf.placeholder(tf.float32, shape=(None, 2), name='x-input')
y_ = tf.placeholder(tf.float32, shape=(None, 1), name='y_input')
w1 = tf.Variable(tf.random_normal([2, 1], stddev=1, seed=1))
y = tf.matmul(w1, x)
loss_less = 10
loss_more = 1
loss = tf.reduce_sum(tf.where(tf.greater(y, y_), (y - y_) * loss_more, (y_ - y) * loss_less))    # 自定义损失函数
train_step = tf.train.AdamOptimizer(0.001).minimize(loss)
# 生成数据和标签
rdm = RandomState(1)
data_size = 128
X = rdm.rand(data_size, 2)

Y = [[X1+X2+rdm.rand()/10.0-0.05] for (X1, X2) in X]
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    steps = 1000
    for i in range(steps):
        start = (i * batch_size) % data_size
        end = (i * batch_size) % data_size + batch_size
        #print(X[start: end])
        #print(Y[start: end])
        #sess.run(train_step, feed_dict={x: X[start:end], y_: Y[start:end]})      # 为何会出错？
    print (sess.run(w1))
                     

[[-0.8113182]
 [ 1.4845988]]


In [27]:

# 指数衰减学习率：tf.train.exponential_decay
# 正则化L1， L2
w = tf.constant([[1.0, 2.0], [-3.0, 4.0]])
with tf.Session() as sess:
    print(sess.run(tf.contrib.layers.l1_regularizer(.5)(w)))   # L1
    print(sess.run(tf.contrib.layers.l2_regularizer(.5)(w)))   # L2

5.0
7.5


In [33]:
# 将变量加入到集合中，便于搭建深度网络
import tensorflow as tf

def get_weight(shape, lambda0):
    var = tf.Variable(tf.random_normal(shape), dtype=tf.float32)
    # 将损失项加入集合，add_to_collection函数的第一个参数是集合的名称
    tf.add_to_collection('losses', tf.contrib.layers.l2_regularizer(lambda0)(var))
    return var

x = tf.placeholder(tf.float32, shape=(None, 2))
y_ = tf.placeholder(tf.float32, shape=(None, 1))
batch_size = 8
layer_dimension = [2, 10, 10, 10, 1]
n_layers = len(layer_dimension)
cur_layer = x
in_demension = layer_dimension[0]

# 创建5层神经网络。
for i in range(1, n_layers):
    out_demension = layer_dimension[i]   #出度，即这层网络的神经元与下一层网络相连的神经元个数。
    w = get_weight([in_demension, out_demension], 1e-3) 
    print(w)
    bias = tf.Variable(tf.constant(0.1, shape=[out_demension]))   # 偏置
    cur_layer = tf.nn.relu(tf.matmul(cur_layer, w) + bias)   # 每层神经网络的输出
    in_demension = layer_dimension[i]
    
mse_loss = tf.reduce_mean(tf.square(cur_layer - y_))    # 模型在训练集中的损失
tf.add_to_collection('losses', mse_loss)

# 从集合中获取所有的损失，将这些损失相加即为最终的损失
loss = tf.add_n(tf.get_collection('losses'))
print(loss)

<tf.Variable 'Variable_33:0' shape=(2, 10) dtype=float32_ref>
<tf.Variable 'Variable_35:0' shape=(10, 10) dtype=float32_ref>
<tf.Variable 'Variable_37:0' shape=(10, 10) dtype=float32_ref>
<tf.Variable 'Variable_39:0' shape=(10, 1) dtype=float32_ref>
Tensor("AddN_2:0", shape=(), dtype=float32)


In [4]:
# 滑动平均模型
# 它能使模型在测试集上的表现更健壮
import tensorflow as tf
v1 = tf.Variable(0, dtype=tf.float32)    # 定义一个用于计算滑动平均的变量，计算滑动平均的变量必须是实数！
step = tf.Variable(0, trainable=False)
ema = tf.train.ExponentialMovingAverage(0.99, step)
maintrain_averages_op = ema.apply([v1])   # 意思是只给变量v1使用滑动平均
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    print(sess.run([v1, ema.average(v1)]))        # v1刚初始化后的值与其滑动平均值
    
    sess.run(tf.assign(v1, 5))     # 更新v1的值到5
    sess.run(maintrain_averages_op)
    print(sess.run([v1, ema.average(v1)]))   
    
    sess.run(tf.assign(step, 1000))
    sess.run(tf.assign(v1, 10))
    sess.run(maintrain_averages_op)
    print(sess.run([v1, ema.average(v1)]))
    
    sess.run(maintrain_averages_op)
    print(sess.run([v1, ema.average(v1)]))
    

[0.0, 0.0]
[5.0, 4.5]
[10.0, 4.555]
[10.0, 4.60945]
